In [2]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

### Creating a dataframe


In [3]:
ser=pd.Series([1,2,3])
df=pd.DataFrame({'a':[1,2,3],'b':['Apple','cow','banana']})
print(ser)
print(df)

0    1
1    2
2    3
dtype: int64
   a       b
0  1   Apple
1  2     cow
2  3  banana


- in the above we have labels like 0,1,2 we can acess through labels if we need we can change through index,

In [4]:
print(ser[0])
ser=pd.Series([1,2,3],index=['a','b','c'])
print(ser)

1
a    1
b    2
c    3
dtype: int64


### Loading data from files(csv,txt,json)

In [5]:
df=pd.read_csv(r"E:\Pavan work\Data files\retailmax_customer_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Customer_ID            200 non-null    object 
 1   Gender                 200 non-null    object 
 2   Age                    200 non-null    int64  
 3   Device_Type            200 non-null    object 
 4   Day_of_Week            200 non-null    object 
 5   Time_Spent_on_App      190 non-null    float64
 6   Total_Purchase_Amount  190 non-null    float64
 7   Used_Discount_Code     200 non-null    object 
dtypes: float64(2), int64(1), object(5)
memory usage: 12.6+ KB


In [6]:
#print(df)

- if we need entire dataframe then we use to_string() method.

In [7]:
#print(df.to_string())

In [8]:
df.head(5) # gives the first 5 rows of the dataframe

,Customer_ID,Gender,Age,Device_Type,Day_of_Week,Time_Spent_on_App,Total_Purchase_Amount,Used_Discount_Code
0,C1000,Male,49,Desktop,Saturday,44.9,130.01,No
1,C1001,Female,56,Desktop,Thursday,38.8,74.41,Yes
2,C1002,Male,49,Mobile,Sunday,25.7,66.34,No
3,C1003,Male,21,Mobile,Wednesday,26.6,195.47,No
4,C1004,Male,47,Desktop,Friday,12.7,127.36,Yes


## Preprocessing data

### Empty cells
- we can drop or fill the empty cells.

In [9]:
x=df['Time_Spent_on_App'].mean()
df.fillna({'Time_Spent_on_App': x}, inplace=True)

- we use inplace=true to change the original dataframe if we dont use that then it will create a copy of the data frame and changes are not reflected in the original dataframe.

In [10]:
df.dropna(inplace=True) # to remove the null values
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 190 entries, 0 to 199
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Customer_ID            190 non-null    object 
 1   Gender                 190 non-null    object 
 2   Age                    190 non-null    int64  
 3   Device_Type            190 non-null    object 
 4   Day_of_Week            190 non-null    object 
 5   Time_Spent_on_App      190 non-null    float64
 6   Total_Purchase_Amount  190 non-null    float64
 7   Used_Discount_Code     190 non-null    object 
dtypes: float64(2), int64(1), object(5)
memory usage: 13.4+ KB


In [11]:

data = {
        'name': ['John Doe', 'jane smith', 'BOB JOHNSON', '  alice brown  ', 'Charlie-Wilson', None],
        'email': ['john@email.com', 'JANE@EMAIL.COM', 'bob@invalid', 'alice@email', 'charlie.wilson@company.co.uk', ''],
        'phone': ['123-456-7890', '(555) 123-4567', '9876543210', '+1-800-555-0123', 'invalid_phone', '555.123.4567'],
        'date_of_birth': ['01/15/1990', '1985-12-25', '03-20-1978', '1995/05/10', '12.30.1980', 'invalid_date'],
        'salary': ['$50,000', '75000', '$120,500.50', '45k', '€60,000', 'not_a_number'],
        'postal_code': ['12345', '90210-1234', 'K1A 0A6', 'SW1A 1AA', '12345-', '123'],
        'gender': ['M', 'Female', 'male', 'f', 'MALE', ''],
        'percentage': ['85%', '92.5%', '0.78', '88', '95.2%', 'invalid']
        }
df = pd.DataFrame(data)
print(df.to_string())

              name                         email            phone date_of_birth        salary postal_code  gender percentage
0         John Doe                john@email.com     123-456-7890    01/15/1990       $50,000       12345       M        85%
1       jane smith                JANE@EMAIL.COM   (555) 123-4567    1985-12-25         75000  90210-1234  Female      92.5%
2      BOB JOHNSON                   bob@invalid       9876543210    03-20-1978   $120,500.50     K1A 0A6    male       0.78
3    alice brown                     alice@email  +1-800-555-0123    1995/05/10           45k    SW1A 1AA       f         88
4   Charlie-Wilson  charlie.wilson@company.co.uk    invalid_phone    12.30.1980       €60,000      12345-    MALE      95.2%
5             None                                   555.123.4567  invalid_date  not_a_number         123            invalid


In [ ]:
cleaned_df = df.copy()
def clean_text(df,col):

    df[col]=df[col].str.strip()
    df[col]=df[col].str.title()
    df[col]=df[col].replace('',np.nan)
    df[col]=df[col].str.replace('-',' ').str.title()
def clean_email(df):

    df["email"]=df["email"].str.strip() 
    df['email']=df['email'].str.lower()
    email_pattern=r'^[a-z0-9._%-+]+@[a-z0-9_%-+.]+\.[a-z]{2,}$'
    mask=df['email'].notna() & df['email'].str.match(email_pattern)
    df.loc[~mask,'email']=np.nan

def clean_phone(df):
    df['phone'] = df['phone'].fillna('')
    df['phone'] = df['phone'].replace(r'\D', '', regex=True)
    def check_validity(phone_number):
        if not phone_number or pd.isna(phone_number):
            return np.nan
        if len(phone_number) == 10:
            return f"{phone_number[0:3]}-{phone_number[3:6]}-{phone_number[6:]}"
        elif len(phone_number) == 11 and phone_number.startswith('1'):
            return f"{phone_number[1:4]}-{phone_number[4:7]}-{phone_number[7:]}"
        else:
            return np.nan
    df['phone'] = df['phone'].apply(check_validity)

def clean_date_of_birth(df):
    def parse_date(date_string):
        for fmt in ('%m/%d/%Y','%Y-%m-%d','%m-%d-%Y','%Y/%m/%d','%m.%d.%Y'):
            try:
                return pd.to_datetime(date_string, format=fmt)
            except ValueError:
                continue
        return pd.NaT
    df['date_of_birth']=df['date_of_birth'].apply(parse_date)


def clean_salary(df):
    def parse_salary(salary_string):
        if pd.isna(salary_string):
            return np.nan
        salary_string=salary_string.replace('$','').replace('€','').replace(',','').strip().lower()
        if salary_string.endswith('k'):
            try:
                return float(salary_string[:-1])*1000
            except ValueError:
                return np.nan
        try:
            return float(salary_string)
        except ValueError:
            return np.nan
    df['salary']=df['salary'].apply(parse_salary)


def clean_gender(df):
    df['gender']=df['gender'].str.strip().str.title()
    df['gender']=df['gender'].replace({'Male':'M','Female':'F'})


def clean_percentage(df):
    def parse_percentage(perc_string):
        if pd.isna(perc_string):
            return np.nan
        perc_string=perc_string.strip().replace('%','')
        try:
            value=float(perc_string)
            if value<1:
                return value*100
            return value
        except ValueError:
            return np.nan
    df['percentage']=df['percentage'].apply(parse_percentage)

def clean_postal_code(df):
    # Example: keep only valid US ZIP or Canadian postal codes, else set to NaN
    def parse_postal(postal):
        if pd.isna(postal):
            return np.nan
        postal = postal.strip()
        us_zip = re.compile(r'^\d{5}(-\d{4})?$')
        ca_zip = re.compile(r'^[A-Za-z]\d[A-Za-z][ -]?\d[A-Za-z]\d$')
        uk_zip = re.compile(r'^[A-Za-z]{1,2}\d[A-Za-z\d]?\s*\d[A-Za-z]{2}$')
        if us_zip.match(postal) or ca_zip.match(postal) or uk_zip.match(postal):
            return postal
        return np.nan
    df['postal_code'] = df['postal_code'].apply(parse_postal)

def clean_all(df):
    df.replace('', np.nan, inplace=True)
    clean_text(df, 'name')
    clean_email(df)
    clean_phone(df)
    clean_date_of_birth(df)
    clean_salary(df)
    clean_gender(df)
    clean_percentage(df)
    clean_postal_code(df)

clean_all(cleaned_df)
print(cleaned_df.to_string())
    



             name                         email         phone date_of_birth    salary postal_code gender  percentage
0        John Doe                john@email.com  123-456-7890    1990-01-15   50000.0       12345      M        85.0
1      Jane Smith                jane@email.com  555-123-4567    1985-12-25   75000.0  90210-1234      F        92.5
2     Bob Johnson                           NaN  987-654-3210    1978-03-20  120500.5     K1A 0A6      M        78.0
3     Alice Brown                           NaN  800-555-0123    1995-05-10   45000.0    SW1A 1AA      F        88.0
4  Charlie Wilson  charlie.wilson@company.co.uk           NaN    1980-12-30   60000.0         NaN      M        95.2
5            None                           NaN  555-123-4567           NaT       NaN         NaN    NaN         NaN


In [32]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   name           5 non-null      object        
 1   email          3 non-null      object        
 2   phone          5 non-null      object        
 3   date_of_birth  5 non-null      datetime64[ns]
 4   salary         5 non-null      float64       
 5   postal_code    6 non-null      object        
 6   gender         5 non-null      object        
 7   percentage     5 non-null      float64       
dtypes: datetime64[ns](1), float64(2), object(5)
memory usage: 516.0+ bytes
